In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config import account_id

In [ ]:
# Replace {account_id} with the actual account ID of the player
# Define the API URL for a player's match history
url = "https://api.opendota.com/api/players/{account_id}/wordcloud".format(account_id=account_id)

stopwords = pd.read_csv("stopwords.csv")
stopwords2=pd.read_csv("filter-wordlist.csv")

In [ ]:
# Send a GET request to the API endpoint to retrieve the player's match history
response = requests.get(url)
total=requests.get(url).json()
wins=requests.get(url+"?win").json()

In [ ]:
#pandas dataframe creation for total matches vs. wins
total_df=pd.DataFrame(total).dropna(how="any")
wins_df=pd.DataFrame(wins).dropna(how="any")
total_df

In [ ]:
#cleaning coumn names for the wins dataframe
wins_df=wins_df.rename(columns={"my_word_counts": "my_word_counts (win)"})
wins_df

In [ ]:
#merging of total and win dataframes
merge_df=pd.merge(total_df, wins_df, left_index=True, right_index=True)
merge_df=merge_df[["my_word_counts", "my_word_counts (win)","all_word_counts_x"]]

#cleaning of the combined dataframe by applying common-word filters
merge_df["Percent Win"]=100*merge_df["my_word_counts (win)"]/merge_df["my_word_counts"].round(0)
merge_df=merge_df[~merge_df.index.isin(stopwords["stopwords"])]
merge_df=merge_df[~merge_df.index.isin(stopwords2["stopwords"])]

merge_df=merge_df.rename(columns={"all_word_counts_x":"all_word_counts"})

merge_df.head(50)

In [ ]:
#sorting the dataframe by percent win, etc.
#min_cutoff is the minimum number of words required to display on a chart. If your chart is too messy, increase this value; else you can decrease it.
min_cutoff=40
sorted_df=merge_df[(merge_df["my_word_counts"]>min_cutoff)].sort_values(["Percent Win", "my_word_counts"], ascending=False)
sorted_df

In [ ]:
#bar chart
sorted_df[["Percent Win", "my_word_counts"]].plot(kind="bar", color=["red", "blue"], title="Wordcloud vs. Win Percentage", ylim=(0,100), position=0, figsize=(20,10), ylabel="% Win/Word Count", grid=True, legend=True)


In [ ]:
#scatter chart
sorted_df.plot(kind="scatter", x= "Percent Win", y="my_word_counts", ylim=(0, 400), xlim=(50, 100), title="Wordcloud vs. Win Percentage", figsize=(20,10), grid=True, legend=True)
for i, txt in enumerate(sorted_df.index):
   plt.annotate(txt, (sorted_df["Percent Win"].iat[i]+0.5, sorted_df["my_word_counts"].iat[i]))
